# 实时API使用指南：语音与文本交互

本指南将介绍如何使用实时API进行语音和文本交互，包括模型功能（如音频和文本生成、函数调用）所需的事件流程，以及如何理解实时会话的状态。
ref：

## 准备工作

首先，我们需要安装必要的Python库来使用实时API：

In [ ]:
# 安装所需库
%pip install websocket-client
%pip install soundfile
%pip install numpy
%pip install pyaudio

## 实时会话基础概念

实时会话是模型与连接的客户端之间的有状态交互，主要包含以下组件：
- **会话（Session）对象**：控制交互参数，如使用的模型、生成输出的语音等配置
- **对话（Conversation）**：表示当前会话中生成的用户输入项和模型输出项
- **响应（Responses）**：模型生成的音频或文本项，会被添加到对话中
- **输入音频缓冲区与WebSocket**：处理音频输入输出的机制

## 建立WebSocket连接

要使用实时API，首先需要通过WebSocket建立连接。以下是连接所需的信息：
- URL: wss://api.stepfun.com/v1/realtime
- 查询参数: model（需连接的实时模型ID，当前仅支持 step-1o-audio step-audio-2 step-audio-2-mini）
- 请求头: Authorization: Bearer YOUR_API_KEY

In [ ]:
import os
import json
import websocket
import threading
import time
import base64
import pyaudio
import logging

# 配置日志记录 
logger = logging.getLogger()# 获取根日志器
logger.handlers = []#清除所有现有处理器

#  添加文件处理器
fhandler = logging.FileHandler(filename='./log/02_realtime.log', mode='a')
file_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')  # 文件日志保留详细信息
fhandler.setFormatter(file_formatter)
logger.addHandler(fhandler)

# 添加控制台（Notebook）处理器
console_handler = logging.StreamHandler()  
console_formatter = logging.Formatter('%(message)s')  # 只输出消息内容
console_handler.setFormatter(console_formatter)
logger.addHandler(console_handler)

# 记录所有>= DEBUG的日志，只显示 >= INFO 的日志
logger.setLevel(logging.DEBUG)
fhandler.setLevel(logging.DEBUG)
console_handler.setLevel(logging.INFO)

# 从环境变量获取API密钥
STEPFUN_API_KEY = os.environ.get("STEPFUN_API_KEY")
if not STEPFUN_API_KEY:
    # 如果环境变量中没有设置，可以在这里直接填写（不推荐在生产环境中这样做）
    # STEPFUN_API_KEY = "your_api_key_here"
    raise ValueError("请设置STEPFUN_API_KEY环境变量或直接在代码中填写API密钥")

# WebSocket 配置（当前仅支持 step-1o-audio step-audio-2 step-audio-2-mini）
url = "wss://api.stepfun.com/v1/realtime?model=step-1o-audio"
headers =[f"Authorization: Bearer {STEPFUN_API_KEY}"]

# 初始化pyaudio和音频流
pa = pyaudio.PyAudio()
audio_stream = None 

# 定义消息队列（供后面handle_audio_events读取）
received_messages = []  # 存储WebSocket收到的消息
received_msg_lock = threading.Lock()  # 消息队列的线程安全锁

# 定义回调函数
def on_open(ws):
    logging.info("已连接到服务器")

def on_message(ws, message):
    global audio_stream
    try:
        data = json.loads(message)
        msg_type = data["type"]
        logging.info(f"事件类型: {msg_type}")
        logging.debug(f"事件内容：{json.dumps(data, indent=2, ensure_ascii=False)}")  # 格式化打印完整内容
        
        # 创建session时输出session_id
        if msg_type =="session.created":
            print(data)
            session_id = data["session"]["id"]
            logging.info(f"Session ID: {session_id}")
        
        if msg_type =="session.updated":
            logging.debug(f"session已更新：{json.dumps(data, indent=2, ensure_ascii=False)}")  # 格式化打印完整内容
        # 音频处理
        with received_msg_lock:
            received_messages.append(data)  # 将消息添加到队列中，供后续处理

        if msg_type == "response.audio.delta":
            # 步骤1：提取并解码Base64音频数据
            audio_b64 = data.get("delta", "")
            audio_binary = base64.b64decode(audio_b64)
            logging.debug(f"解码后音频数据长度: {len(audio_binary)} 字节")

            # 步骤2：初始化音频流（首次收到音频时启动）
            if not audio_stream or audio_stream.is_stopped():
                audio_stream = pa.open(
                    format=pyaudio.paInt16,  # 16位音频（TTS服务常见格式）
                    channels=1, # 单声道
                    rate=24000, # 采样率
                    output=True,  # 输出模式（播放）
                    frames_per_buffer=1024  # 缓冲区大小（平衡延迟和稳定性）
                )
                logging.info("音频播放流已初始化，开始播放...")

            # 步骤3：实时写入音频数据到流（播放）
            pure_audio = audio_binary  # 已为纯PCM数据，直接使用
            audio_stream.write(pure_audio) # 写入流播放（即时生效）
        if msg_type == "response.audio.done":
            if audio_stream:
                audio_stream.stop_stream()
                audio_stream.close()
                audio_stream = None
                logging.info("音频播放完成，流已关闭")
            logging.info("音频响应完成")

        # 文本处理
        if msg_type == 'response.audio_transcript.delta':
            logging.info(f"————{data['delta']}")

        # 函数处理
        if msg_type == 'response.function_call_arguments.delta':
            logging.info(f"…………{data['delta']}") 

        if msg_type == 'response.done':
            logging.info(f"响应完成:{json.dumps(data, indent=2, ensure_ascii=False)}")
        if msg_type =='error':
            logging.error(f"❌ 服务器返回错误：{data['error']}")
        else:
            pass
    except Exception as e:
        logging.error(f"❌ 解析消息时出错: {e}")


def on_error(ws, error):
    logging.error(f"发生错误: {error}")

def on_close(ws, close_status_code, close_msg):
    logging.error(f"连接已关闭，状态码: {close_status_code}, 信息: {close_msg}")

# 初始化WebSocket应用
ws = websocket.WebSocketApp(
    url,
    header=headers,
    on_open=on_open,
    on_message=on_message,
    on_error=on_error,
    on_close=on_close
)

# 在后台线程中运行WebSocket连接
ws_thread = threading.Thread(target=ws.run_forever)
ws_thread.daemon = True
ws_thread.start()

# 等待连接建立
time.sleep(2)

Websocket connected
已连接到服务器
事件类型: session.created
Session ID: 019a71bedc72732a9b5a51bc47989854


{'event_id': '0f808e27-1079-481e-9a0a-b539235c6881', 'type': 'session.created', 'session': {'id': '019a71bedc72732a9b5a51bc47989854', 'object': 'realtime.session', 'model': 'step-1o-audio', 'modalities': ['text', 'audio'], 'voice': 'jingdiannvsheng', 'input_audio_format': 'pcm16', 'output_audio_format': 'pcm16', 'volume_ratio': 1}}


事件类型: session.updated
事件类型: conversation.item.created
事件类型: response.created
事件类型: response.output_item.added
事件类型: conversation.item.created
事件类型: response.content_part.added
事件类型: response.audio.done
音频响应完成
事件类型: response.audio_transcript.done
事件类型: response.done
响应完成:{
  "event_id": "0f4b5aa8-7342-4754-8a43-3e15b97a4256",
  "type": "response.done",
  "response": {
    "id": "a656797f-d0bf-4cf3-9a60-53cde56dc279",
    "object": "realtime.response",
    "status": "completed",
    "output": [
      {
        "id": "56614a47-c4ab-4524-95d0-b22a1ed65166",
        "type": "message",
        "status": "in_progress",
        "role": "assistant",
        "content": [
          {
            "type": "audio",
            "audio": "",
            "transcript": ""
          }
        ],
        "object": "realtime.item"
      }
    ],
    "usage": {
      "total_tokens": 325,
      "input_tokens": 303,
      "output_tokens": 22,
      "input_token_details": {
        "cached_tokens": 256,
      

**USER_INPUT**
* **`conversation.item.created`** *# 确认用户输入 Item 已加入历史*

--- **RESPONSE 流程** ---

* **`conversation.item.created`** *# (连接上一个 Item，作为新 Item 写入历史的信号)*

* **`response.created`** *# **[步骤 1: 启动]** 新的推理流程开始*
    * **[response id]** *# 此次 Response 的唯一标识*

    --- **Item Generation (输出) 阶段** ---

    * **`response.output_item.added`** *# **[步骤 2: 结构诞生]** Item 骨架通知 (同一 Item ID)*
        * **`response.content_part.added`** *# Content Part #1 流开始*
            * ***# 媒体流式传输阶段***
            * `response.audio.transcript.delta`
            * `response.audio.delta`
            * ...
            * `response.audio.transcript.done`
            * `response.audio.done`
            
        * **`response.content_part.done`** *# Content Part #1 完成*
        
    * **`response.output_item.done`** *# **[步骤 3: 内容完成]** Item 所有内容传输完成*
        
* **`response.done`** *# **[步骤 4: 流程结束]** 整个 Response 流程最终结束*
    * `统计出token用量`  *# 计费流程*

## 更新会话配置

连接建立后，服务器会发送`session.created`事件，表示会话已准备就绪。我们可以通过`session.update`事件更新会话配置。这里添加内置的web检索工具`web_search`

In [13]:

event = {
    "type": "session.update",
    "session": {
        "instructions": "你需要礼貌地回复",  # system prompt
        "voice": "qingchunshaonv",
        "turn_detection": {
            "type": "server_vad"
        },
        "tools": [
            {
                "type": "web_search",
                "function": {
                    "description": "搜索工具",
                    "options": {
                        "top_k": 1,  # 指定返回结果的数量
                        "timeout_seconds": 1  # 指定搜索的超时时间
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "generate_horoscope",
                    "description": "根据用户的星座生成今日运势。",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "sign": {
                                "type": "string",
                                "description": "需要查询运势的星座。",
                                "enum": [
                                    "白羊座", "金牛座", "双子座", "巨蟹座", "狮子座", "处女座",
                                    "天秤座", "天蝎座", "射手座", "摩羯座", "水瓶座", "双鱼座"
                                ]
                            }
                        },
                        "required": ["sign"]
                    }
                }
            }
        ]
    }
}

ws.send(json.dumps(event))
# 等待服务器响应
time.sleep(1)

## 文本交互

使用文本与模型交互需要创建文本对话项并请求模型生成响应。

In [ ]:
# 发送文本消息
def send_text_message(ws, text):
    if not ws.sock or not ws.sock.connected:
        print("WebSocket连接未建立")
        return
    
    event = {
        "type": "conversation.item.create",
        "item": {
            "type": "message",
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": text,
                }
            ]
        }
    }
    
    ws.send(json.dumps(event))
    print(f"已发送文本消息: {text}")

# 请求生成文本响应
def request_text_response(ws):
    if not ws.sock or not ws.sock.connected:
        print("WebSocket连接未建立")
        return
    
    event = {
        "type": "response.create",
        "response": {
            "modalities": ["text"]  # 仅生成文本响应
        }
    }
    
    ws.send(json.dumps(event))
    print("已请求文本响应")

# 示例：发送问题并请求响应
# send_text_message(ws, "我是处女座，我的今日运势如何")
# send_text_message(ws, "查询浙江和上海2023年的GDP")
send_text_message(ws, "你好")
time.sleep(1)  # 等待消息被处理
request_text_response(ws)

# 等待响应完成
time.sleep(10)

已发送文本消息: 查询浙江和上海2023年的GDP
已请求文本响应


如果请求成功但听不见流式音频播放，更新一下portaudio

## 音频交互

实时API的强大功能之一是支持语音到语音的交互，无需中间的文本转语音或语音转文本步骤。

### 向服务器流式传输音频输入

要向服务器流式传输音频输入，可以使用input_audio_buffer.append客户端事件。该事件要求您通过套接字向实时API发送Base64编码的音频字节块。每个块的大小不能超过15 MB。

输入块的格式可以为整个会话配置，也可以为每个响应单独配置。

会话级：session.update中的session.input_audio_format
响应级：response.create中的response.input_audio_format

In [13]:
import base64
import soundfile as sf
import numpy as np


# 将音频文件转换为Base64编码字符串
def audio_to_base64(file_path):
    with open(file_path, "rb") as audio_file:
        audio_bytes = audio_file.read()
        return base64.b64encode(audio_bytes).decode('utf-8')

# 添加音频数据到输入缓冲区event
def send_input_audio_buffer_append(ws,filepath):
  audio_base64 = audio_to_base64(filepath)
  input_audio_buffer_append_msg = {
        # "event_id": event_id,
        "type": "input_audio_buffer.append",
        "audio": audio_base64
    }
  fullAudio=json.dumps(input_audio_buffer_append_msg)
  ws.send(fullAudio)

# 提交音频缓冲区并请求响应
def commit_audio_and_request_response(ws):
    if not ws.sock or not ws.sock.connected:
        print("WebSocket连接未建立")
        return
    
    # 提交音频缓冲区
    ws.send(json.dumps({"type": "input_audio_buffer.commit"}))
    print("已提交音频缓冲区")
    time.sleep(1)
    
    # 请求音频响应
    event = {
        "type": "response.create",
        "response": {
            "modalities": ["audio", "text"]  # 同时生成音频和文本
        }
    }
    
    ws.send(json.dumps(event))
    print("已请求音频响应")

# 从文件发送音频（示例）
def send_audio_from_file(ws, file_path):
    try:
        # 读取音频文件并发送
        send_input_audio_buffer_append(ws,file_path)

        # 提交并请求响应
        commit_audio_and_request_response(ws)
        return True
    except Exception as e:
        print(f"发送音频时出错: {e}")
        return False

# 请将下面的文件路径替换为实际的音频文件路径
send_audio_from_file(ws, "./media/02_realtime_input.wav")

# 等待响应完成
time.sleep(5)

已提交音频缓冲区
已请求音频响应


## 处理音频输出

要处理模型返回的音频，需要监听`response.audio.delta`事件，收集音频数据块并将其组合成完整的音频文件。


In [14]:
# 处理音频输出的示例
import threading

# 用于存储接收到的音频数据
audio_buffer = []
audio_lock = threading.Lock()

# 自定义音频事件处理函数
def handle_audio_events(ws):
    global audio_buffer
    while ws.sock and ws.sock.connected:
        try:
            # 检查新消息
            if received_messages:
                with audio_lock:
                    msg = received_messages.pop(0)
                
                if msg['type'] == 'response.audio.delta':
                    # 存储接收到的音频数据
                    with audio_lock:
                        audio_buffer.append(msg['delta'])
                    print(f"收到音频块，当前缓冲区大小: {len(audio_buffer)}")
                
                elif msg['type'] == 'response.audio.done':
                    print("音频传输完成，正在保存文件...")
                    save_audio_buffer("./media/02_realtime_output_audio.wav")
                    with audio_lock:
                        audio_buffer = []  # 重置缓冲区
            
            time.sleep(0.1)
        except Exception as e:
            print(f"处理音频事件时出错: {e}")
            break

# 保存音频缓冲区到文件
def save_audio_buffer(file_path):
    global audio_buffer
    try:
        with audio_lock:
            if not audio_buffer:
                print("音频缓冲区为空")
                return
            
            # 合并所有音频块并解码保存
            full_audio_base64 = ''.join(audio_buffer)
            audio_bytes = base64.b64decode(full_audio_base64)       
            pcm16 = np.frombuffer(audio_bytes, dtype=np.int16)  
            sf.write(file_path, pcm16, 24000, subtype='PCM_16')  
            print(f"音频已保存到 {file_path}")
    except Exception as e:
        print(f"保存音频时出错: {e}")

# 启动音频处理线程（传入ws实例，供线程判断连接状态）
audio_event_thread = threading.Thread(target=handle_audio_events, args=(ws,))
audio_event_thread.daemon = True
audio_event_thread.start()
logging.info("音频处理线程已启动，等待接收音频数据...")

# 等待响应完成
time.sleep(5)

音频处理线程已启动，等待接收音频数据...


收到音频块，当前缓冲区大小: 1
收到音频块，当前缓冲区大小: 2
收到音频块，当前缓冲区大小: 3
收到音频块，当前缓冲区大小: 4
收到音频块，当前缓冲区大小: 5


收到音频块，当前缓冲区大小: 6
收到音频块，当前缓冲区大小: 7
收到音频块，当前缓冲区大小: 8
收到音频块，当前缓冲区大小: 9
收到音频块，当前缓冲区大小: 10
收到音频块，当前缓冲区大小: 11
收到音频块，当前缓冲区大小: 12
收到音频块，当前缓冲区大小: 13
收到音频块，当前缓冲区大小: 14
收到音频块，当前缓冲区大小: 15
收到音频块，当前缓冲区大小: 16
收到音频块，当前缓冲区大小: 17
收到音频块，当前缓冲区大小: 18
收到音频块，当前缓冲区大小: 19
收到音频块，当前缓冲区大小: 20
收到音频块，当前缓冲区大小: 21
收到音频块，当前缓冲区大小: 22
收到音频块，当前缓冲区大小: 23
收到音频块，当前缓冲区大小: 24
收到音频块，当前缓冲区大小: 25
收到音频块，当前缓冲区大小: 26
收到音频块，当前缓冲区大小: 27
收到音频块，当前缓冲区大小: 28
收到音频块，当前缓冲区大小: 29
收到音频块，当前缓冲区大小: 30
收到音频块，当前缓冲区大小: 31
收到音频块，当前缓冲区大小: 32
收到音频块，当前缓冲区大小: 33
收到音频块，当前缓冲区大小: 34
收到音频块，当前缓冲区大小: 35
收到音频块，当前缓冲区大小: 36
收到音频块，当前缓冲区大小: 37
收到音频块，当前缓冲区大小: 38
收到音频块，当前缓冲区大小: 39
收到音频块，当前缓冲区大小: 40
收到音频块，当前缓冲区大小: 41
收到音频块，当前缓冲区大小: 42
收到音频块，当前缓冲区大小: 43
收到音频块，当前缓冲区大小: 44
收到音频块，当前缓冲区大小: 45
收到音频块，当前缓冲区大小: 46
收到音频块，当前缓冲区大小: 47
收到音频块，当前缓冲区大小: 48
收到音频块，当前缓冲区大小: 49
收到音频块，当前缓冲区大小: 50
收到音频块，当前缓冲区大小: 51
收到音频块，当前缓冲区大小: 52
收到音频块，当前缓冲区大小: 53
收到音频块，当前缓冲区大小: 54
收到音频块，当前缓冲区大小: 55
收到音频块，当前缓冲区大小: 56
音频传输完成，正在保存文件...
音频已保存到 ./media/02_realtime_output_audio.wav


### *添加conversation

可以创建包含完整音频录制的对话消息。使用conversation.item.create客户端事件创建带有input_audio内容的消息。

（注：input_audio_buffer.append 事件会有回复，conversation.item.create事件不会有回复，只起到添加上下文的作用 😊💦）

In [ ]:
fullAudio=audio_to_base64("./media/realtime-input.wav")

event = {
    "type": "conversation.item.create",
    "item": {
        "type": "message",
        "role": "user",
        "content": [
            {
                "type": "input_audio",
                "audio": fullAudio,
            }
        ],
    },
}
 
ws.send(json.dumps(event))

## 关闭连接

使用完毕后，应正确关闭WebSocket连接：

In [10]:
# 关闭WebSocket连接
if ws.sock and ws.sock.connected:
    ws.close()
    print("连接已关闭")

# 等待线程结束
ws_thread.join(timeout=5)

连接已关闭


## 总结

本指南介绍了如何使用实时API进行语音和文本交互，包括：
- 建立WebSocket连接
- 管理会话生命周期
- 发送文本消息并获取响应
- 流式传输音频和发送完整音频消息
- 处理和保存模型返回的音频

通过监听不同类型的事件，您可以实时了解会话状态并提供良好的用户体验。实时API支持语音到语音的直接交互，大大降低了语音界面的延迟。